In [1]:
import torch
torch.set_grad_enabled(False)  # Disable gradient calculation for inference

In [2]:
import sys

sys.path.append('/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/repos/segfm3d-nora')
sys.path.append('/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/repos/nnunetv2')

In [3]:
config = {
    "DATA_DIR": "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data",
    "RESULTS_DIR": "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/results",
    "SAM_CKPT_PATH": "/nfs/norasys/notebooks/camaret/SAM-Med3D/ckpt/sam_med3d_turbo_bbox_cvpr.pth",
    "SAM_REPO_DIR": "/nfs/norasys/notebooks/camaret/SAM-Med3D",
    "ONNX_MODEL_PATH": "/nfs/norasys/notebooks/camaret/model_inference/models/sammed3d.onnx",
    "NNINT_CKPT_DIR": "/nfs/norasys/notebooks/camaret/model_checkpoints/nnint",
}

In [4]:
%load_ext autoreload
%autoreload 1

In [5]:
import os
import torch
from nnInteractive.inference.inference_session import nnInteractiveInferenceSession
import os.path


nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [6]:
args_for_network = {
    "arch_class_name": "dynamic_network_architectures.architectures.unet.ResidualEncoderUNet",
    "arch_kwargs": {
        "n_stages": 6,
        "features_per_stage": [32, 64, 128, 256, 320, 320],
        "conv_op": "torch.nn.modules.conv.Conv3d",
        "kernel_sizes": [
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
        ],
        "strides": [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]],
        "n_blocks_per_stage": [1, 3, 4, 6, 6, 6],
        "n_conv_per_stage_decoder": [1, 1, 1, 1, 1],
        "conv_bias": True,
        "norm_op": "torch.nn.modules.instancenorm.InstanceNorm3d",
        "norm_op_kwargs": {"eps": 1e-05, "affine": True},
        "dropout_op": None,
        "dropout_op_kwargs": None,
        "nonlin": "torch.nn.LeakyReLU",
        "nonlin_kwargs": {"inplace": True},
    },
    "arch_kwargs_req_import": ["conv_op", "norm_op", "dropout_op", "nonlin"],
    "input_channels": 8,
    "output_channels": 2,
    "allow_init": True,
    "deep_supervision": True,
}

In [7]:
from nnunetv2.utilities.get_network_from_plans import get_network_from_plans
network = get_network_from_plans(**args_for_network)

In [8]:
total_params = sum(p.numel() for p in network.parameters())
print(f"Total parameters: {total_params:,}")


Total parameters: 102,355,818


In [9]:
from nnunetv2.utilities.get_network_from_plans import get_network_from_plans
network = get_network_from_plans(**args_for_network).cuda()
orig_checkpoint_dir = '/nfs/norasys/notebooks/camaret/model_checkpoints/nnint/nnInteractive_v1.0/'
newer_checkpoint_dir = "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/new-dataset/"
if os.path.exists(newer_checkpoint_dir):
    chkpt_dict = torch.load(os.path.join(newer_checkpoint_dir, 'fold_0/checkpoint_final.pth'), weights_only=False)
else:
    chkpt_dict = torch.load(os.path.join(orig_checkpoint_dir, 'fold_0/checkpoint_final.pth'), weights_only=False)

network.load_state_dict(chkpt_dict['network_weights'])

<All keys matched successfully>

In [10]:
%aimport src.training.transforms

In [11]:
from src.training.npzdataset import NPZNoCacheDataset
from src.training.transforms import CropToNonzeroImage
from src.training.transforms import NormalizeSingleImageTransformNumpy
from src.training.transforms import ChooseClass
from src.training.transforms import CropAndAddBBox
from src.training.transforms import AddEmptyChan
from src.training.transforms import WrapMONAI
from src.training.transforms import MergeImageAndInteractionAndSeg
from batchgeneratorsv2.transforms.utils.deep_supervision_downsampling import (
    DownsampleSegForDSTransform,
)
from monai.transforms import Resized
from batchgeneratorsv2.transforms.utils.compose import ComposeTransforms

only_2d_bbox = False

test_transforms = [
            CropToNonzeroImage(),
            NormalizeSingleImageTransformNumpy(),
            ChooseClass(),
            CropAndAddBBox(),
            AddEmptyChan(),
            WrapMONAI(Resized(
                keys=["image", "segmentation", "bbox_chan"], spatial_size=(192, 192, 192),
                mode=["trilinear", "nearest", "nearest"]
            )),
            MergeImageAndInteractionAndSeg(),
        ]
test_transform = ComposeTransforms(test_transforms)
train_transforms = test_transforms + [
            DownsampleSegForDSTransform(
            ds_scales=[
                [1.0, 1.0, 1.0],
                [0.5, 0.5, 0.5],
                [0.25, 0.25, 0.25],
                [0.125, 0.125, 0.125],
                [0.0625, 0.0625, 0.0625],
            ])]
train_transform =ComposeTransforms(train_transforms)

In [12]:
from src.training.npzdataset import NPZPytorchDataset, NPZDataset
from src.training.utils import fast_ids_to_deterministic_floats
import numpy as np

from glob import glob
parent_folder = '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/3D_train_npz_random_10percent_16G/'

to_exclude = ['Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg07.npz',
       'Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg24.npz',
       'Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg37.npz',
       'CT/CT_Abdomen1K/CT_Abdomen1K_Case_00931.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_002_sc.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_000_sc.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_001_sc.npz']
all_npz_file_paths = glob(os.path.join(parent_folder, '**/*.npz'), recursive=True)

cleaned_file_paths = np.array(sorted([f for f in all_npz_file_paths if not any(e in f for e in to_exclude)]))

train_mask = fast_ids_to_deterministic_floats(cleaned_file_paths) < 0.8
train_files = cleaned_file_paths[train_mask]
test_files = cleaned_file_paths[~train_mask]
train_set = NPZNoCacheDataset(train_files, transform=train_transform)
test_set = NPZNoCacheDataset(test_files, transform=test_transform)

In [13]:
n_jobs = int(os.environ['SLURM_CPUS_PER_TASK'])

In [14]:
import torch
train_loader = torch.utils.data.DataLoader(
    train_set,
    shuffle=True,
    drop_last=True,
    num_workers=n_jobs,
    batch_size=2,
)

In [15]:

test_loader = torch.utils.data.DataLoader(
    test_set,
    shuffle=False,
    drop_last=False,
    num_workers=n_jobs,
    batch_size=2,
)

In [16]:
from tqdm.autonotebook import tqdm, trange
import torch
from src.training.model_wrap import ModelPrevSegAndClickWrapper
from src.training.utils import defer_keysignal_with_grad

wrapped_network = ModelPrevSegAndClickWrapper(network, n_max_clicks=1)

optim_network = torch.optim.AdamW(
    wrapped_network.parameters(), lr=1e-4, weight_decay=1e-5
)

with custom forward print shape


In [17]:
from copy import deepcopy
import shutil
out_dir = '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/new-dataset/'#'/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/try_from_nb_all_data_competition_bbox/'
try:
    shutil.copytree(orig_checkpoint_dir, out_dir)
except FileExistsError:
    print(f"path {out_dir} exists already, fine")
trained_chkpt_path = os.path.join(out_dir, 'fold_0/checkpoint_final.pth')
assert os.path.exists(trained_chkpt_path)


trained_chkpt = deepcopy(chkpt_dict)

path /nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/new-dataset/ exists already, fine


In [18]:
accumulate_batch_size = 16
n_batches_accumulate = accumulate_batch_size // 2

In [19]:
from fvcore.nn import FlopCountAnalysis

# Get a batch from the dataloader iterator
train_loader_iter = iter(train_loader)
batch = next(train_loader_iter)
input_tensor = batch["image"].cuda()

flops = FlopCountAnalysis(wrapped_network, input_tensor)

print(f"FLOPs: {flops.total()}")  # In raw FLOPs (can divide by 1e9 for GFLOPs)

Unsupported operator aten::alias encountered 312 time(s)
Unsupported operator aten::leaky_relu_ encountered 116 time(s)
Unsupported operator aten::add_ encountered 52 time(s)
Unsupported operator aten::avg_pool3d encountered 10 time(s)
Unsupported operator aten::diff encountered 1 time(s)
Unsupported operator aten::ne encountered 1 time(s)
Unsupported operator aten::meshgrid encountered 1 time(s)
Unsupported operator aten::sub encountered 1 time(s)
Unsupported operator aten::sum encountered 2 time(s)
Unsupported operator aten::pow encountered 1 time(s)
Unsupported operator aten::sqrt encountered 1 time(s)
Unsupported operator aten::lt encountered 1 time(s)
Unsupported operator aten::mul encountered 1 time(s)
Unsupported operator aten::__or__ encountered 2 time(s)


FLOPs: 9887643426816


In [20]:
from src.training.utils import compute_binary_dsc
from IPython.display import display
import pandas as pd
from datetime import datetime

n_epochs = 25
moving_dsc = None
all_mean_dscs = []
i_batch = 0
optim_network.zero_grad()
for i_epoch in trange(n_epochs):
    epoch_mean_dscs = []
    for batch in (pbar := tqdm(train_loader)):
        with defer_keysignal_with_grad():
            targets = batch["segmentation"][0][:, 0].long().cuda()
            out = wrapped_network(batch["image"].cuda())
            # assuming deep supervision
            pred = out[0]
            assert pred[:, 0].shape == targets.shape
            cent = torch.nn.functional.cross_entropy(pred, targets)
            binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:, 0])
            assert binary_pred.shape == targets.shape
            eps = 1e-3
            intersect = torch.sum(binary_pred * targets, dim=(1, 2, 3))
            sum_pred = torch.sum(binary_pred, dim=(1, 2, 3))
            sum_targets = torch.sum(targets, dim=(1, 2, 3))
            mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
            loss = cent + 10 * (1 - mean_dsc)
            loss = loss / n_batches_accumulate  ## account for accumulation
            loss.backward()
            i_batch += 1
            if (i_batch % n_batches_accumulate) == 0:
                all_grad_finite = all(
                    [
                        (p.grad.isfinite().all().item())
                        for group in optim_network.param_groups
                        for p in group["params"]
                        if (p.grad is not None)
                    ]
                )
                if all_grad_finite and torch.isfinite(loss).item():
                    optim_network.step()
                else:
                    print("loss/grads not finite, not updating!")
                optim_network.zero_grad()
            else:
                pass  ## accumulate grad...
        epoch_mean_dscs.append(mean_dsc.item())
        if torch.isfinite(loss).item():
            moving_dsc = (
                mean_dsc if moving_dsc is None else (moving_dsc * 0.98 + mean_dsc * 0.02)
            )
        pbar.set_postfix(dict(moving_dsc=moving_dsc.item()))
        print(mean_dsc.item())
        print(cent.item())
        print()
    print("mean dsc", np.nanmean(epoch_mean_dscs))
    all_mean_dscs.extend(epoch_mean_dscs)
    print(f"Epoch {i_epoch}")
    if torch.isfinite(loss).item():
        filename_time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")[:-3]
        trained_chkpt["network_weights"] = wrapped_network.orig_network.state_dict()
        if torch.isfinite(loss).item():
            torch.save(trained_chkpt, trained_chkpt_path)
            print(f"Checkpoint saved to {trained_chkpt_path}")
        current_ckpt_filename = trained_chkpt_path.replace(
            "/checkpoint_final.pth", f"/checkpoint_{i_epoch}_{filename_time_str}.pth"
        )
        torch.save(trained_chkpt, current_ckpt_filename)
        print(f"Checkpoint also saved to {current_ckpt_filename}")

    test_results = []
    for batch in (pbar := tqdm(test_loader)):
        targets = batch["segmentation"][:, 0].long().cuda()
        out = wrapped_network(batch["image"].cuda())
        # assuming deep supervision
        pred = out[0]
        assert pred[:, 0].shape == targets.shape
        cent = torch.nn.functional.cross_entropy(pred, targets)
        binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:, 0])
        assert binary_pred.shape == targets.shape
        eps = 1e-3
        intersect = torch.sum(binary_pred * targets, dim=(1, 2, 3))
        sum_pred = torch.sum(binary_pred, dim=(1, 2, 3))
        sum_targets = torch.sum(targets, dim=(1, 2, 3))
        mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
        test_results.append(
            dict(
                hard_dsc=compute_binary_dsc((binary_pred > 0.5), targets),
                soft_dsc=((2 * intersect + eps) / (sum_pred + sum_targets + eps))
                .cpu()
                .numpy()
                .mean(),
                cent=cent.item(),
            )
        )
    display(pd.DataFrame(test_results).mean())
    test_result_filepath = os.path.join(
        os.path.split(trained_chkpt_path)[0],
        f"test_results_{i_epoch}_{filename_time_str}.csv",
    )
    # save test results
    pd.DataFrame(test_results).to_csv(test_result_filepath)
    print(f"Saved test results to {test_result_filepath}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1193 [00:00<?, ?it/s]

0.8594262003898621
0.0002466946607455611

0.8792238235473633
0.009743480943143368

0.7520604133605957
0.010547947138547897

0.9208256006240845
0.005537268240004778



KeyboardInterrupt: 

In [ ]:
trained_chkpt["network_weights"] = wrapped_network.orig_network.state_dict()
torch.save(trained_chkpt, trained_chkpt_path)
print(f"Checkpoint saved to {trained_chkpt_path}")

In [ ]:

trained_chkpt["network_weights"] = wrapped_network.orig_network.state_dict()
torch.save(trained_chkpt, trained_chkpt_path)
print(f"Checkpoint saved to {trained_chkpt_path}")

In [ ]:
newer_checkpoint_dir

In [ ]:
# look at examples etc.
# now try click changes, grad clip? gradient accum?
# different sampling?
# old loki
# http://10.217.3.189:8888/lab?token=7fe7920caa53ef1ca025abdc8c4fdaedc532ecf340635f2f


# nero
# http://10.231.0.162:8888/lab?token=1e5d7331c18592a8cb58ced34b88232869e16ea21a98b4c9

In [ ]:
test_results = []
for batch in (pbar := tqdm(test_loader)):
    targets = batch["segmentation"][:, 0].long().cuda()
    out = wrapped_network(batch["image"].cuda())
    # assuming deep supervision
    pred = out[0]
    assert pred[:, 0].shape == targets.shape
    cent = torch.nn.functional.cross_entropy(pred, targets)
    binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:, 0])
    assert binary_pred.shape == targets.shape
    eps = 1e-3
    intersect = torch.sum(binary_pred * targets, dim=(1, 2, 3))
    sum_pred = torch.sum(binary_pred, dim=(1, 2, 3))
    sum_targets = torch.sum(targets, dim=(1, 2, 3))
    mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
    test_results.append(
        dict(
            hard_dsc=compute_binary_dsc((binary_pred > 0.5), targets),
            soft_dsc=((2 * intersect + eps) / (sum_pred + sum_targets + eps)).cpu().numpy().mean(),
            cent=cent.item()
        )
    )

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
finite_dscs = np.array(all_mean_dscs)[np.isfinite(np.array(all_mean_dscs))]

In [ ]:
plt.plot(np.correlate(finite_dscs, np.ones(100)/100))

In [ ]:
display(pd.DataFrame(test_results).mean())

In [ ]:
display(pd.DataFrame(test_results).mean())

In [35]:
1

In [ ]:
1